# Dashboard Data Wrangling

In [53]:
import pandas as pd
import numpy as np

In [54]:
# Importing kaggle survey questions through 24
survey_data = pd.read_csv("../data/Raw/kaggle_survey_2020_responses.csv").loc[:,:"Q24"]

C:\Users\jacob\miniconda3\envs\532\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Question Key For Easy Reference

* Q3: residence country (query)
* Q4: highest level of formal education aquired. (tile plot)
* Q5: current or most recent job title (query).
* Q6: Programming years. (query)
* Q7: Programming Languages Used on a regular basis. (bar plot)
* Q8: Programming Languages Recommended to learn first for Aspiring Data Scientist (tile plot)
* Q17: Machine learning algorithms used on a regular basis. (barplot)
+ Q24: Yearly Compensation (USD) (card)



In [55]:
stage_1_data = survey_data[survey_data.columns[survey_data.columns.str.contains("Q[3-8]|Q17|Q24",regex=True)]]

In [56]:
# Removing responses from individuals who reported that they had never written code before
remove_phrase = ["I have never written code"]
stage_1_data = stage_1_data[~stage_1_data["Q6"].isin(remove_phrase)]

In [57]:
# income wrangling

# Filtering out responses below 6000 USD A year as these were likely not full time workers
stage_2_data = stage_1_data.query('Q24 != "$0-999" & Q24 != "1,000-1,999" & Q24 != "2,000-2,999" & Q24 != "3,000-3,999" & Q24 != "4,000-4,999" & Q24 != "5,000-5,999"')
stage_2_data = stage_2_data.iloc[1:]

# Reworking Binning, Splitting Data For Sorting
stage_2_data['Q24'] = stage_2_data['Q24'].replace("> $500,000", "500,000-600,000")
stage_2_data['Q24'] = stage_2_data['Q24'].replace("$500,000", "500,000-600,000")
stage_2_data['Q24'] = stage_2_data['Q24'].replace(",", "", regex=True)
stage_2_data[['lower','higher']] = stage_2_data.Q24.str.split("-", expand=True)
stage_2_data["lower"] = pd.to_numeric(stage_2_data["lower"])
stage_2_data["higher"] = pd.to_numeric(stage_2_data["higher"])

In [58]:
stage_2_data.to_csv("..\data\Processed\general_processed_data.csv", index = False)

In [59]:
stage_2_data.to_csv('general_processed_data.csv', index = False)

# Language Barplot wrangling 

In [60]:
# Melting all mentions of ML methods down to one column for plotting
language_barplot_survey_data = stage_1_data.iloc[1:].filter(regex = 'Q[3-7]', axis =1)
language_columns = language_barplot_survey_data.columns[language_barplot_survey_data.columns.str.contains("Q7", regex=True)].tolist()
id_columns = language_barplot_survey_data.columns[language_barplot_survey_data.columns.str.contains("Q3|Q4|Q5|Q6", regex=True)].tolist()
lang_barplot_data = language_barplot_survey_data.melt(id_vars= id_columns, value_vars=language_columns, var_name="prog_lang", value_name="selected_lang").dropna()

In [61]:
lang_barplot_data.to_csv('../data/Processed/lang_barplot_data.csv', index = False)

# ML Methods Barplot wrangling 

In [62]:
# Melting all mentions of ML methods down to one column for plotting
machine_learning_barplot_survey_data = stage_1_data.iloc[1:].filter(regex = 'Q[3456]|Q17', axis =1)
ml_columns = machine_learning_barplot_survey_data.columns[machine_learning_barplot_survey_data.columns.str.contains("Q17", regex=True)].tolist()
id_columns = machine_learning_barplot_survey_data.columns[machine_learning_barplot_survey_data.columns.str.contains("Q3|Q4|Q5|Q6", regex=True)].tolist()
ml_barplot_data = machine_learning_barplot_survey_data.melt(id_vars= id_columns, value_vars=ml_columns, var_name="ml_method", value_name="selected_ml_method").dropna()


# Shortening Names With Specific Pairings
ml_barplot_data = ml_barplot_data.replace({'Transformer Networks (BERT, gpt-3, etc)' : 'Transformer Networks',
                                          "Dense Neural Networks (MLPs, etc)" : "Dense Neural Networks",
                                          "Gradient Boosting Machines (xgboost, lightgbm, etc)" : "Gradient Boosting Machines",
                                          "Decision Trees or Random Forests" : "Decision Trees",
                                          "None" : "I do not use ML Methods"
                                          })

# Replacing instances of Networks with Nets For Brevity
ml_barplot_data['selected_ml_method'] = ml_barplot_data['selected_ml_method'].str.replace('Networks', 'Nets', regex = True)

In [63]:
ml_barplot_data.to_csv('../data/Processed/ml_barplot_data.csv', index = False)

# Education vs. Recommended Language Dot Plot

In [64]:
fluctuation_plot_survey_data = stage_1_data.iloc[1:].filter(regex = 'Q[3|4|5|6|8]', axis = 1)

In [65]:
# Replacing Chunky Names with Simpler Names
fluctuation_plot_survey_data = fluctuation_plot_survey_data.replace({'Some college/university study without earning a bachelor’s degree' : 'Some college/university study',
                                                                      "No formal education past high school" : "Highschool Diploma"})

In [66]:
fluctuation_plot_survey_data.to_csv('../data/Processed/Fluctuations_plot_data.csv', index = False)